### IMPORTING PACKAGES

In [172]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import sklearn
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score


pd.set_option('mode.chained_assignment', None)

(1527, 2)


,comments,label
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^,1
4,watch?v=vtaRGgvGtWQ Check this out .,1


### READING DATA

In [ ]:
spam = pd.read_csv('D:\\Praxis\\ML\\Data\\spam_new.csv')
spam.head()

In [114]:
# Distribution of Ham and Spam comments
spam.label.value_counts()

1    778
0    749
Name: label, dtype: int64

In [115]:
# dtypes
spam.dtypes

comments    object
label        int64
dtype: object

### Train test split

In [118]:
train, test = train_test_split(spam, test_size=0.2,random_state=1234,stratify=spam['label'])

In [119]:
test.label.value_counts()

1    156
0    150
Name: label, dtype: int64

### DATA PREPROCESSING

In [120]:
train["comments"]= train["comments"].astype(str)

In [123]:
train['comments'] = train['comments'].str.replace('\W', ' ') # Removes punctuation
train['comments'] = train['comments'].str.lower()
#X_train.head(4)

In [124]:
sum(train.comments.isnull())

0

In [177]:
#X_train.head()

In [176]:
#X_train['comments'].str.split()

### Getting unique words in the dataset and storing them

In [125]:
train['comments'] = train['comments'].str.split()

vocabulary = []

for sms in train['comments']:
    for word in sms:
        vocabulary.append(word)

vocabulary = list(set(vocabulary))

### Finding the number of times a word occured in a comment

In [126]:
word_counts_per_comment = {word: [0] * len(train['comments']) for word in vocabulary}

In [127]:
for index, comments in enumerate(train['comments']):
    for word in comments:
        word_counts_per_comment[word][index] += 1

In [128]:
word_counts = pd.DataFrame(word_counts_per_comment)
#word_counts.head()

In [62]:
word_counts.shape

(1221, 3653)

In [130]:
#word_counts.isnull()

### Concatenating the word count frame to train data

In [131]:
train_set = pd.concat([train.reset_index(drop=True), word_counts.reset_index(drop=True)], axis=1)
#train_set

In [132]:
#train_set.isnull()

In [134]:
train_set.shape

(1221, 3655)

### Computing probabilities and frequencies

In [135]:
# Filtering spam and ham comments
spam_comments = train_set[train_set['label'] == 1]
ham_comments = train_set[train_set['label'] == 0]

p_spam = len(spam_comments) / len(train_set) # probability of comment being spam
p_ham = len(ham_comments) / len(train_set) # probability of comment being ham

n_words_per_spam_comment = spam_comments['comments'].apply(len)
n_spam = n_words_per_spam_comment.sum() # total_number_of_words_in_spam_comments

n_words_per_ham_comment = ham_comments['comments'].apply(len)
n_ham = n_words_per_ham_comment.sum() # total_number_of_words_in_spam_comments

n_vocabulary = len(vocabulary) # number of unique words in vocabulary

In [ ]:
alpha = 1 # laplace smoothing; useful to come out of 0 frequency problem

### CALCULATING POSTERIOR PROBABILITIES

In [136]:
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# P(word | spam) = (n(word|spam) + alpha) / (number of spam words + alpha * vocabulary)

for word in vocabulary:
    n_word_given_spam = spam_comments[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam

    n_word_given_ham = ham_comments[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

### CREATING A CLASSIFIER FUNCTION

In [189]:
def classify(comment,p_spam,p_ham,parameters_spam,parameters_ham):

    label = 0
    comment = re.sub('\W', ' ', comment)
    comment = comment.lower().split()

    p_spam_given_comment = p_spam # initially it is set to the PRIOR probability of spam comment
    p_ham_given_comment = p_ham
    
    for word in comment:
        if word in parameters_spam:
            p_spam_given_comment *= parameters_spam[word]   # using property of conditional independence

        if word in parameters_ham: 
            p_ham_given_comment *= parameters_ham[word]

    #print('P(Spam|comment):', p_spam_given_comment)
    #print('P(Ham|comment):', p_ham_given_comment)

    if p_ham_given_comment > p_spam_given_comment:
        label = 0
    elif p_ham_given_comment < p_spam_given_comment:
        label = 1
    else:
        label = 1 # We want to be cautious. Even if its a tie, we classify it as a spam comment
    return label

### CROSS VALIDATION

In [ ]:
from sklearn.model_selection import RepeatedKFold

In [196]:
def five_cold_CV(data):    
    kf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=None) 
    accuracy_list = []

    for train_index, test_index in kf.split(data):
        
        train, test = data.iloc[train_index], data.iloc[test_index]    
    
        train["comments"]= train["comments"].astype(str)
        #train['label'] = y_train['label']
        train['comments'] = train['comments'].str.replace('\W', ' ') # Removes punctuation
        train['comments'] = train['comments'].str.lower()

        # unique words in the trainset

        train['comments'] = train['comments'].str.split()

        vocabulary = []

        for sms in train['comments']:
            for word in sms:
                vocabulary.append(word)

        vocabulary = list(set(vocabulary))

        word_counts_per_comment = {word: [0] * len(train['comments']) for word in vocabulary}

        for index, comments in enumerate(train['comments']):
            for word in comments:
                word_counts_per_comment[word][index] += 1

        word_counts = pd.DataFrame(word_counts_per_comment)
        word_counts.head()

        train_set = pd.concat([train.reset_index(drop=True), word_counts.reset_index(drop=True)], axis=1)

        # Isolating spam and ham comments first
        spam_comments = train_set[train_set['label'] == 1]
        ham_comments = train_set[train_set['label'] == 0]

        # P(Spam) and P(Ham)
        p_spam = len(spam_comments) / len(train_set)
        p_ham = len(ham_comments) / len(train_set)

        # N_Spam
        n_words_per_spam_comment = spam_comments['comments'].apply(len)
        n_spam = n_words_per_spam_comment.sum()

        # N_Ham
        n_words_per_ham_comment = ham_comments['comments'].apply(len)
        n_ham = n_words_per_ham_comment.sum()

        # N_Vocabulary
        n_vocabulary = len(vocabulary)

        # Laplace smoothing
        alpha = 1

        # Initiate parameters
        parameters_spam = {unique_word:0 for unique_word in vocabulary}
        parameters_ham = {unique_word:0 for unique_word in vocabulary}

        i = 0

        # Calculate parameters
        for word in vocabulary:
            n_word_given_spam = spam_comments[word].sum() # spam_comments already defined
            #print("{0}- {1},{2}".format(word,n_word_given_spam,i))
            i = i +1
            p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
            parameters_spam[word] = p_word_given_spam

            n_word_given_ham = ham_comments[word].sum() # ham_comments already defined
            p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
            parameters_ham[word] = p_word_given_ham
                
        test["comments"]= test["comments"].astype(str)
        
        test['comments'] = test['comments'].str.replace('\W', ' ') # Removes punctuation
        test['comments'] = test['comments'].str.lower()
                
        test['prediction'] = test['comments'].apply(classify, args=(p_spam,p_ham,parameters_spam,parameters_ham))
        
        tn, fp, fn, tp = confusion_matrix(test['label'], test['prediction']).ravel()
        
        accuracy = (tp+tn)/(tn+ fp+ fn+ tp)
        accuracy_list.append(accuracy)
        
    return np.mean(accuracy)

In [197]:
CV_score = five_cold_CV(train)
CV_score

0.9139344262295082

In [193]:
from functools import partial


### RUNNING THE MODEL ON THE TEST DATA

In [198]:
test['comments'] = test['comments'].str.replace('\W', ' ') # Removes punctuation
test['comments'] = test['comments'].str.lower()

test['prediction'] = test['comments'].apply(classify,args=(p_spam,p_ham,parameters_spam,parameters_ham))

### CONFUSION MATRIX

In [199]:
confusion_matrix(test['label'], test['prediction'])

array([[138,  12],
       [  5, 151]], dtype=int64)

In [200]:
tn, fp, fn, tp = confusion_matrix(test['label'], test['prediction']).ravel()

#### SENSITIVITY, SPECIFICITY, PRECISION, ACCURACY all have values above 0.92

In [201]:
sensitivity_tpr = tp/(fn+tp)
sensitivity_tpr

0.967948717948718

In [202]:
specificity_tnr = tn/(fp+tn)
specificity_tnr

0.92

In [203]:
precision = tp/(tp + fp)
precision

0.9263803680981595

In [204]:
accuracy = (tp+tn)/(tn+ fp+ fn+ tp)
print(accuracy)

0.9444444444444444


### Running the model on WordPress

### Importing packages

In [205]:
from wordpress_xmlrpc import Client
from wordpress_xmlrpc.methods.posts import *
from wordpress_xmlrpc.methods.users import *
from wordpress_xmlrpc.methods.comments import *
from wordpress_xmlrpc.methods.pages import *
 
from collections import Counter

### Function to retrieve comments in a post

In [183]:
def get_all_comments_per_post(client, post_id):

    data = {'filter' : post_id,
            'number': 2000,
            'status': 'approve'}
 
    resp = client.call(GetComments(data))
    
    return resp

#### The following code iterates over posts and then comments in each post, checks if they are spam and then deletes them

In [223]:
from wordpress_xmlrpc.methods import posts

wp = Client(r'https://mydataspace.wordpress.com/xmlrpc.php', 'username', 'password')
print ('Searching for Comments...')

posts_in_blog = wp.call(posts.GetPosts())

for i in range(0, len(posts_in_blog)):
    
    print("Checking in {}...".format(str(posts_in_blog[i])))
    post_id = int(posts_in_blog[i].id)
    
    comments = get_all_comments_per_post(wp,post_id)
    comments_string = []
    
    for j in range(0,len(comments)):
        
        comments_string.append(str(comments[j]))
        if classify(comments_string[j],p_spam,p_ham,parameters_spam,parameters_ham):
            print("deleting {}".format(comments_string[j]))
            wp.call(DeleteComment(comments[j].id))
            
print("Check complete...")

Searching for Comments...
Checking in An unique case where Decision Tree outperforms the Random Forest...
Checking in Finding Similar Players using Clustering Algorithm...
Check complete...
